* fix exponential interpolation Nan issue
* mark "testing set" of model.model
* mark "def loss" of model.model

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

import tkinter
from tkinter import*
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)
from tkinter import filedialog
%gui tk

import FFT
import compensation
import module_4D
# import modeling_white_card
import model
import white_card_std

# from modeling_white_card import modeling_white
# from modeling import modeling
import interpolation

In [2]:
def data_loading():
    path = filedialog.askdirectory()
    print(path)
    return path

In [3]:
path = data_loading()

D:/Data_store/AMS_7341/122420 FPCB_ZnD_calibration/#rigid_flex_01


In [4]:
#sensor serial number
sensor_number = path.split('/')[-1]
print('sensor_number:', sensor_number)

sensor_number: #rigid_flex_01


# Loading data and FFT transform

In [5]:
# load training data
A = 0
B = 30000
df_0C = FFT.FFT_transform(pd.read_csv(path + '/select_data_calibration_0C.csv')[A:B].reset_index(drop=True))
df_10C = FFT.FFT_transform(pd.read_csv(path + '/select_data_calibration_10C.csv')[A:B].reset_index(drop=True))
df_20C = FFT.FFT_transform(pd.read_csv(path + '/select_data_calibration_20C.csv')[A:B].reset_index(drop=True))
df_30C = FFT.FFT_transform(pd.read_csv(path + '/select_data_calibration_30C.csv')[A:B].reset_index(drop=True))
train_data_list = [df_0C, df_10C, df_20C, df_30C]

# Load testing data
df_test_20C = FFT.FFT_transform(pd.read_csv(path + '/select_data_blind_test_20C.csv')[A:B].reset_index(drop=True))
df_test_30C = FFT.FFT_transform(pd.read_csv(path + '/select_data_blind_test_30C.csv')[A:B].reset_index(drop=True))
df_test_20C_2 = FFT.FFT_transform(pd.read_csv(path + '/select_data_blind_test_20C_2.csv')[A:B].reset_index(drop=True))
df_test_30C_2= FFT.FFT_transform(pd.read_csv(path + '/select_data_blind_test_30C_2.csv')[A:B].reset_index(drop=True))
df_test_30C_3= FFT.FFT_transform(pd.read_csv(path + '/select_data_blind_test_30C_3.csv')[A:B].reset_index(drop=True))
test_data_list = [df_test_20C, df_test_30C, df_test_20C_2, df_test_30C_2, df_test_30C_2]

In [6]:
print('df_0C:', df_0C.shape)
print('df_10C:', df_10C.shape)
print('df_20C:', df_20C.shape)
print('df_30C:', df_30C.shape)
print('-------------------------------------')
print('df_test_20C:', df_test_20C.shape)
print('df_test_30C:', df_test_30C.shape)
print('df_test_20C_2:', df_test_20C_2.shape)
print('df_test_30C_2:', df_test_30C_2.shape)
print('df_test_30C_3:', df_test_30C_3.shape)

df_0C: (225, 70)
df_10C: (225, 70)
df_20C: (225, 70)
df_30C: (225, 70)
-------------------------------------
df_test_20C: (40, 70)
df_test_30C: (40, 70)
df_test_20C_2: (40, 70)
df_test_30C_2: (40, 70)
df_test_30C_3: (15, 70)


## White card model setting

In [7]:
# white card paremeter setting
# name = '(0,10,30,60ppm model)(hybrid interpolation)(670nm)(w1s2)'
name = 'test'
model_name = Ridge(alpha=10)                    #regression model
channel = '670nm #1'                       #regression channel
degree_white_card = 1                                   #polynimial degree
humidity_step = ['RH40%', 'RH50%', 'RH60%', 'RH70%', 'RH80']
white_card_humidity_feature = False

In [8]:
df_all = pd.concat(train_data_list).reset_index(drop=True)
df_test = pd.concat([df_10C]).reset_index(drop=True)

y_train = df_all[channel]
x_train = df_all[['Temperature', 'Humidity']]
y_test = df_test[channel]
x_test = df_test[['Temperature', 'Humidity']]

os.chdir(path) 
white_card = model.model(name=name,
                         sensor_number=sensor_number,
                         channel=channel,
                         x_train=x_train, 
                         y_train=y_train, 
                         x_test=x_test, 
                         y_test=y_test, 
                         model_name=model_name, 
                         degree=degree_white_card,
                         step=humidity_step,                         
                         ppm=-23,
                         humidity_feature=white_card_humidity_feature)

white_intercept, white_coef = white_card.regression()
white_card.save_white_card_side_coef()
# white_intercept, white_coef = white_card.coef()

In [9]:
# add white card std column
for train_data_comp in train_data_list: 
    white_card_std.create_white_card_std_column(train_data_comp, white_intercept, white_coef, degree_white_card)
for test_data_comp in test_data_list: 
    white_card_std.create_white_card_std_column(test_data_comp, white_intercept, white_coef, degree_white_card) 

In [10]:
# Compensation process
# white_card_std_data = df_10C                       #for constant compensation function
phage_side = 2                                #for compensation
white_card_side = 1                           #for compensation
# comp_channel = "583nm"

for train_data_comp in train_data_list: 
    compensation.compensation_AS7341(train_data_comp, phage_side=phage_side, white_card_side=white_card_side, compChannel=channel)
    
for test_data_comp in test_data_list: 
    compensation.compensation_AS7341(test_data_comp, phage_side=phage_side, white_card_side=white_card_side, compChannel=channel)

## Sensor model setting

In [11]:
sensor_humidity_feature = True
model_name = Ridge(alpha=10)                    #regression model
shift = 0                                       #raw data signal shifting
multiple = 1                                    #raw data ppm multiple
train_channel = '670nm #1'                   #training channel
test_channel = '670nm #1'                    #testing channel
degree = 2                                      #polynimial degree
output_modify = 1                               #output multiple

In [21]:
# load training data
dk_0C = df_0C.copy().reset_index(drop=True)
dk_10C = df_10C.copy().reset_index(drop=True)
dk_20C = df_20C.copy().reset_index(drop=True)
dk_30C = df_30C.copy().reset_index(drop=True)

# copy testing data
dk_test_20C = df_test_20C.copy().reset_index(drop=True)
dk_test_30C = df_test_30C.copy().reset_index(drop=True)
dk_test_20C_2 = df_test_20C_2.copy().reset_index(drop=True)
dk_test_30C_2 = df_test_30C_2.copy().reset_index(drop=True)
dk_test_30C_3 = df_test_30C_3.copy().reset_index(drop=True)

#setting temperature range
'''dict = {temp. 1 : ([training data1, data2], [blind test data1, data2], white_card_std data)}'''

temp_data_dict = { 

    # n20~10C
    'temp. 1': ([dk_0C, dk_10C, dk_20C, dk_30C], [dk_test_20C, dk_test_30C]),
    'temp. 2': ([dk_0C, dk_10C, dk_20C, dk_30C], [dk_test_20C_2, dk_test_30C_2]),
    'temp. 3': ([dk_0C, dk_10C, dk_20C, dk_30C], [dk_test_30C_3]),

                  } 
    
# implement temp. range loop
for temp_data, temp_blind_test in temp_data_dict.values():
    

    # plot each model 4D data
#             module_4D.plot_4D_multi_model(temp_data, temp_blind_test, channel_list, sensor_number)

    # concate training data        
    df_all = pd.concat(temp_data).reset_index(drop=True)
    df_test = pd.concat(temp_blind_test).reset_index(drop=True)


    # 取0ppm資料
    dk_all_0ppm = df_all[df_all['ppm']<1].copy()

    # 取10ppm資料
    dk_all_10ppm = df_all[(df_all['ppm']>9)&(df_all['ppm']<11)].copy()

    # 取30ppm資料
    dk_all_30ppm = df_all[(df_all['ppm']>29)&(df_all['ppm']<31)].copy()

    # 取60ppm資料
    dk_all_60ppm = df_all[(df_all['ppm']>59)].copy()

    
    dk_data_dict = {
                    '0ppm':(dk_all_0ppm),
                    '10ppm':(dk_all_10ppm),
                    '30ppm':(dk_all_30ppm),
                    '60ppm':(dk_all_60ppm)
                   }
    

    #training model for 0ppm, 10ppm, 60ppm surface
    surf_coef_list = []                           # store ppm surface coefficient
    for ppm, ppm_data in dk_data_dict.items():

        dk_data = ppm_data


        y_train = dk_data[train_channel] + shift 
        x_train = dk_data[['Temperature', 'Humidity']] 

#             channels = train_channel

        #開始計算濃度平面 model by Ridge regression
#         step = humidity_step
        os.chdir(path)              
        loss_list = []
        loss_each_step_list = []
        sensor = model.model(name=name,
                             sensor_number=sensor_number,
                             channel=train_channel,
                             x_train=x_train, 
                             y_train=y_train, 
                             x_test=x_test, 
                             y_test=y_test, 
                             model_name=model_name, 
                             degree=degree,
                             step=humidity_step,
                             output_modify=output_modify,
                             shift=shift,            
                             multiple=multiple,
                             ppm=ppm,
                             humidity_feature=sensor_humidity_feature)
        intercept, coef = sensor.regression()
        sensor.save_sensor_side_coef()
#         sensor.loss()
#         loss_each_step = sensor.loss_each_step()    
#         sensor.plot()
#         sensor.save_plot()
#         sensor.coef()
        
        #補0調整濕度項次數
        if (sensor_humidity_feature == False)&(degree==1):
            coef = np.append(coef, [0] * degree)

        elif (sensor_humidity_feature == False)&(degree==2):
            coef = np.insert(coef, 1, 0)
            coef = np.append(coef, [0] * degree)

        elif (sensor_humidity_feature == False)&(degree==3):
            coef = np.insert(coef, 1, 0)
            coef = np.insert(coef, 3, 0)
            coef = np.insert(coef, 4, 0)
            coef = np.append(coef, [0] * degree)
            
        coefficient = np.insert(coef, 0, intercept)
        surf_coef_list.append(coefficient) 
   
    test_data = df_test[[test_channel, 'Temperature', 'Humidity', 'ppm']]
    ppm_list = []
    for index, row in test_data.iterrows():
        signal, temperature, humidity, _ = row
        signal_0ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[0], x0=temperature, x1=humidity, degree=degree)
        signal_10ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[1], x0=temperature, x1=humidity, degree=degree)
        signal_30ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[2], x0=temperature, x1=humidity, degree=degree)
        signal_60ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[3], x0=temperature, x1=humidity, degree=degree)

#         print(signal_0ppm, signal_10ppm, signal_60ppm)
        
        signal_ppm_dict = {
                             0:signal_0ppm,
                            10:signal_10ppm,
                            30:signal_30ppm,
                            60:signal_60ppm
                           }

        ppm = interpolation.linear_interpolation(signal_ppm_dict, signal, 0, 10, 60)
#         ppm = interpolation.exponential_interpolation(signal_ppm_dict, signal, 0, 10)
#         ppm = interpolation.hybrid_interpolation(signal_ppm_dict, signal, 0, 10, 60)
        ppm_list.append(ppm)

    # RMSE
    rmse_test = np.sqrt(mean_squared_error(ppm_list, test_data['ppm'])).round(2)

    # Interpolation prediction result    
    predict_test_data = test_data.copy()
    predict_test_data['pred_ppm'] = ppm_list
    temp = int(predict_test_data['Temperature'].mean())
    hum_min = int(predict_test_data['Humidity'].min())
    hum_max = int(predict_test_data['Humidity'].max())
    title = '({}%~{}%) ({}) (RMSE={})'.format(hum_min, hum_max, test_channel, rmse_test)
    predict_test_data.iplot(kind='scatter', y=['ppm', 'pred_ppm', 'Humidity', 'Temperature'], title=title)


    # save prediction result
    folderName = '{}'.format(name)
    savePath = os.path.join(os.getcwd(), folderName)
    filename = '({}%~{}%)({})(shift {})(RMSE={})'.format(hum_min, hum_max, test_channel, shift ,rmse_test)
    plt.plot(predict_test_data['Temperature'], 'r')
    plt.plot(predict_test_data['Humidity'], 'b')
    plt.plot(predict_test_data['ppm'], 'g')
    plt.plot(predict_test_data['pred_ppm'], 'orange')
    plt.ylabel('Temperature/ Humidity/ ppm')
    plt.grid(alpha=0.3)
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102),  loc=3, ncol=4, mode="expand")
    plt.title(filename, y=1.15, fontsize=10)
    plt.savefig(savePath + '/' + filename + '.png', dpi=200, bbox_inches='tight')
    plt.close() 
    plt.show()

# Overview 4D data

In [14]:
signal_ppm_dict = {
                            0:signal_0ppm,
                            10:signal_10ppm,
                            30:signal_30ppm,
                            60:signal_60ppm
                           }

In [22]:
# 4D plot
training_data = {
             '00C':df_0C,
             '10C':df_10C,
             '20C':df_20C,
             '30C':df_30C,
                 }

testing_data = {
             '20C_test':df_test_20C,
             '30C_test':df_test_30C,   
             '20C_test_2':df_test_20C_2,
             '30C_test_2':df_test_30C_2, 
             '30C_test_3':df_test_30C_3, 
               }

channel_list=['670nm #1']
module_4D.plot_4D(training_data, testing_data, channel_list, sensor_number)

# channel_list=['440nm #2', '440nm #2_comp']
# module_4D.plot_4D(training_data, testing_data, channel_list, sensor_number)